In [227]:
# To plot inline
%matplotlib inline

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [228]:
data_train = pd.read_csv('train.csv')
print('Number Of Rows: {}'.format(len(data_train)))
print('--------------------------')
data_train.head()

Number Of Rows: 891
--------------------------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [229]:
for x in ['Name','Ticket','Cabin','Embarked','Fare']: del data_train[x]
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,male,22.0,1,0
1,2,1,1,female,38.0,1,0
2,3,1,3,female,26.0,0,0
3,4,1,1,female,35.0,1,0
4,5,0,3,male,35.0,0,0


In [230]:
data_train.Sex = data_train.Sex.astype('category').cat.codes
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,1,22.0,1,0
1,2,1,1,0,38.0,1,0
2,3,1,3,0,26.0,0,0
3,4,1,1,0,35.0,1,0
4,5,0,3,1,35.0,0,0


In [231]:
data_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex               int8
Age            float64
SibSp            int64
Parch            int64
dtype: object

In [232]:
print(data_train.isnull().any())
print('----------------------------------')
print('Number Of Rows: {}'.format(len(data_train)))

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age             True
SibSp          False
Parch          False
dtype: bool
----------------------------------
Number Of Rows: 891


In [260]:
mean_age = np.sqrt(np.sum(np.square(data_train.Age.dropna()))/len(data_train.Age.dropna()))
# mean_age = np.mean(data_train.Age)
data_train.fillna(value=(mean_age),inplace=True) 
data_train.isnull().any()

Pclass        False
Sex           False
Age           False
FamilySize    False
dtype: bool

In [234]:
nmbr = data_train.groupby('Survived').count()['PassengerId']
nmbr

Survived
0    549
1    342
Name: PassengerId, dtype: int64

In [235]:
print('%.2f percent of total people survided.\n%.2f percent people died.'%(nmbr[1]*100/(nmbr[1] + nmbr[0]),nmbr[0]*100/(nmbr[1] + nmbr[0])))

38.38 percent of total people survided.
61.62 percent people died.


In [236]:
data_train['FamilySize'] = data_train.SibSp + data_train.Parch
del data_train['SibSp']
del data_train['Parch']
target = data_train['Survived'].copy() 
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,FamilySize
0,1,0,3,1,22.0,1
1,2,1,1,0,38.0,1
2,3,1,3,0,26.0,0
3,4,1,1,0,35.0,1
4,5,0,3,1,35.0,0


In [237]:
gender_survival = data_train.groupby(['Survived','Sex']).count()['PassengerId']
gender_survival

Survived  Sex
0         0       81
          1      468
1         0      233
          1      109
Name: PassengerId, dtype: int64

In [238]:
pw_survided = gender_survival.iloc[1]*100/(gender_survival.iloc[1] + gender_survival.iloc[0])
pm_died = gender_survival.iloc[2]*100/(gender_survival.iloc[2] + gender_survival.iloc[3])
print('%.2f percent of the total survived are women and %.2f of the total died are men'%(pw_survided,pm_died))

85.25 percent of the total survived are women and 68.13 of the total died are men


In [239]:
del data_train['PassengerId']
del data_train['Survived']
data_train.head()

,Pclass,Sex,Age,FamilySize
0,3,1,22.0,1
1,1,0,38.0,1
2,3,0,26.0,0
3,1,0,35.0,1
4,3,1,35.0,0


In [240]:
X = data_train.as_matrix()
print(X)
y = target.as_matrix()

[[  3.           1.          22.           1.        ]
 [  1.           0.          38.           1.        ]
 [  3.           0.          26.           0.        ]
 ..., 
 [  3.           0.          33.05693828   3.        ]
 [  1.           1.          26.           0.        ]
 [  3.           1.          32.           0.        ]]


In [241]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split,KFold

In [250]:
n_splits = 6

svc_params = {'C':[0.9,1,1.5,10]}

rfc_params = {'n_estimators':[30,40],
              'min_samples_split':[10,20],
              'random_state':[10,15],
              'warm_start':(True,False)
             }

etc_params = {'n_estimators':[20,40],
              'min_samples_split':[10,20,50],
              'random_state':[10,15,30],
              'warm_start':(True,False),
              'criterion' : ('gini','entropy')
             }

gbc_params = {  'learning_rate':[0.1,0.15],
              'n_estimators':[10,50,80],
              'max_depth':[3,8,10],
              'warm_start':(True,False),
              'subsample' : [0.5,0.8,1.0]
             }
dtc_params = {'criterion':('gini','entropy'),
             'splitter':('best','random'),
              'max_depth':[5,10,15,None],
              'min_samples_split':[2,5,10]
             }
vtc_params = {'voting': ('hard','soft'),
             
      }


In [251]:
skf = StratifiedKFold(n_splits=n_splits)

clf1 = SVC()
clf1 = GridSearchCV(clf1,svc_params,cv=skf,verbose=1,n_jobs=2)
clf1.fit(X,y)

clf2 = RandomForestClassifier()
clf2 = GridSearchCV(clf2,rfc_params,cv=skf,verbose=1,n_jobs=2)
clf2.fit(X,y)

clf3 = ExtraTreesClassifier()
clf3 = GridSearchCV(clf3,etc_params,cv=skf,verbose=1,n_jobs=2)
clf3.fit(X,y)

clf4 = GradientBoostingClassifier()
clf4 = GridSearchCV(clf4,gbc_params,cv=skf,verbose=1,n_jobs=2)
clf4.fit(X,y)

clf5 = DecisionTreeClassifier()
clf5 = GridSearchCV(clf5,dtc_params,cv=skf,verbose=1,n_jobs=2)
clf5.fit(X,y)

# models = [x.best_estimator_ for x in [clf1,clf2,clf3,clf4,clf5,clf6]]
# scores = [x.best_score_ for x in [clf1,clf2,clf3,clf4,clf5,clf6]]

    

Fitting 6 folds for each of 4 candidates, totalling 24 fits
Fitting 6 folds for each of 16 candidates, totalling 96 fits


[Parallel(n_jobs=2)]: Done  24 out of  24 | elapsed:    0.6s finished


Fitting 6 folds for each of 72 candidates, totalling 432 fits


[Parallel(n_jobs=2)]: Done  96 out of  96 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Done 172 tasks      | elapsed:    5.4s
[Parallel(n_jobs=2)]: Done 432 out of 432 | elapsed:   14.0s finished


Fitting 6 folds for each of 108 candidates, totalling 648 fits


[Parallel(n_jobs=2)]: Done 307 tasks      | elapsed:   27.0s


Fitting 6 folds for each of 48 candidates, totalling 288 fits


[Parallel(n_jobs=2)]: Done 648 out of 648 | elapsed:  1.0min finished
[Parallel(n_jobs=2)]: Done 288 out of 288 | elapsed:    0.5s finished


GridSearchCV(cv=StratifiedKFold(n_splits=6, random_state=None, shuffle=False),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'criterion': ('gini', 'entropy'), 'splitter': ('best', 'random'), 'max_depth': [5, 10, 15, None], 'min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [257]:
kf = KFold(n_splits=n_splits)
for tr_index,ts_index in kf.split(X,y):
    clf6 = VotingClassifier(estimators = [('svc',clf1.best_estimator_),('rfc',clf2.best_estimator_),('etc',clf3.best_estimator_),('gbc',clf4.best_estimator_),('dtc',clf5.best_estimator_)])
    X_train = X[tr_index]
    X_test = X[ts_index]
    y_train = y[tr_index]
    y_test = y[ts_index]
    
    clf6.fit(X_train,y_train)
    print('Training Score : %0.2f'%clf6.score(X_train,y_train))
    print('Testing Score : %0.2f'%clf6.score(X_test,y_test))

Training Score : 0.89
Testing Score : 0.77
Training Score : 0.89
Testing Score : 0.83
Training Score : 0.88
Testing Score : 0.87
Training Score : 0.88
Testing Score : 0.83
Training Score : 0.88
Testing Score : 0.82
Training Score : 0.88
Testing Score : 0.87


In [325]:
data_test = pd.read_csv('test.csv')
print('Number Of Rows: {}'.format(len(data_test)))
print('---------------------------------------')
data_test.head(3)

Number Of Rows: 418
---------------------------------------


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [326]:
data_submit = pd.read_csv('gender_submission.csv')
print(len(data_submit.PassengerId))
print(len(data_test.PassengerId))
len(data_submit.PassengerId) == len(data_test.PassengerId)

418
418


True

In [327]:
data_test.drop(data_test.columns[[2,7,8,9,10]],axis=1,inplace=True)
len(data_test.PassengerId)

418

In [328]:
test_mean_age = mean_age = np.sqrt(np.sum(np.square(data_train.Age.dropna()))/len(data_train.Age.dropna()))
data_test.fillna(value=test_mean_age,inplace=True)
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch']
data_test.Sex = data_test.Sex.astype('category').cat.codes
data_test.drop(data_test.columns[[0,4,5]],axis=1,inplace=True)
final_X = data_test.as_matrix()
final_X

array([[  3.        ,   1.        ,  34.5       ,   0.        ],
       [  3.        ,   0.        ,  47.        ,   1.        ],
       [  2.        ,   1.        ,  62.        ,   0.        ],
       ..., 
       [  3.        ,   1.        ,  38.5       ,   0.        ],
       [  3.        ,   1.        ,  33.05693828,   0.        ],
       [  3.        ,   1.        ,  33.05693828,   2.        ]])

In [329]:
final_X.shape

(418, 4)

In [330]:
final_y = clf6.predict(final_X)
final_y[:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [331]:
data_submit['Survived2'] = final_y
del data_submit['Survived']
data_submit.rename(columns = {'Survived2':'Survived'},inplace=True)
data_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [332]:
data_submit.to_csv('submission.csv')